<a href="https://colab.research.google.com/github/EAbdelrahman/Player-Performance-Predictor/blob/main/Rate_Model(best_Fw).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, precision_score, \
recall_score, cohen_kappa_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 999
pd.options.display.max_rows = 50 

# importing data 

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving all_Data.xlsx to all_Data.xlsx


In [ ]:
import io
Full_Data = pd.read_excel(io.BytesIO(uploaded['all_Data.xlsx']))

# Handling Data

##Clean the Data

In [ ]:
filtered_by_league=Full_Data[Full_Data.league=='Premier League']
filtered_by_league_NULL=filtered_by_league[filtered_by_league.rating>0]
filtered_by_league_NULL_FW=filtered_by_league_NULL[filtered_by_league_NULL.position=='Attacker']



## Split Data into Target and Features

## Feature Vector

Let's create a feature vector.

In [ ]:
#list(filtered_by_league_NULL_FW.columns)
filtered_by_league_NULL_FW.columns


Index(['Unnamed: 0', 'Unnamed: 1', 'player_id', 'player_name', 'firstname',
       'lastname', 'number', 'position', 'age', 'birth_date', 'birth_place',
       'birth_country', 'nationality', 'height', 'weight', 'injured', 'rating',
       'team_id', 'team_name', 'league_id', 'league', 'season', 'captain',
       'shots/total', 'shots/on', 'goals/total', 'goals/conceded',
       'goals/assists', 'goals/saves', 'passes/total', 'passes/key',
       'passes/accuracy', 'tackles/total', 'tackles/blocks',
       'tackles/interceptions', 'duels/total', 'duels/won',
       'dribbles/attempts', 'dribbles/success', 'fouls/drawn',
       'fouls/committed', 'cards/yellow', 'cards/yellowred', 'cards/red',
       'penalty/won', 'penalty/commited', 'penalty/success', 'penalty/missed',
       'penalty/saved', 'games/appearences', 'games/minutes_played',
       'games/lineups', 'substitutes/in', 'substitutes/out',
       'substitutes/bench'],
      dtype='object')

In [ ]:
features = ['captain','shots/total'
,'shots/on','goals/total','goals/assists','passes/total','passes/key','passes/accuracy','tackles/total','tackles/blocks'
,'tackles/interceptions','duels/total','duels/won','dribbles/attempts','dribbles/success','fouls/drawn','fouls/committed','cards/yellow','cards/yellowred','cards/red'
,'penalty/won','penalty/commited','penalty/success','penalty/missed','games/appearences','games/minutes_played','games/lineups','substitutes/in','substitutes/out'
,'substitutes/bench','results','players__player_id','players__player_name','players__firstname','players__lastname','players__number','players__position'
,'players__age','players__birth_date','players__birth_place','players__birth_country','players__nationality','players__height','players__weight','players__injured'
,'players__rating','players__team_id','players__team_name','players__league_id','players__league','players__season','players__captain','players__shots__total'
,'players__shots__on','players__goals__total','players__goals__conceded','players__goals__assists','players__goals__saves','players__passes__total','players__passes__key'
,'players__passes__accuracy','players__tackles__total','players__tackles__blocks','players__tackles__interceptions','players__duels__total','players__duels__won'
,'players__dribbles__attempts','players__dribbles__success','players__fouls__drawn','players__fouls__committed','players__cards__yellow','players__cards__yellowred'
,'players__cards__red','players__penalty__won','players__penalty__commited','players__penalty__success','players__penalty__missed','players__penalty__saved'
,'players__games__appearences','players__games__minutes_played','players__games__lineups','players__substitutes__in'
,'players__substitutes__out','players__substitutes__bench']


x=filtered_by_league_NULL_FW.iloc[:,20:53]
X=x.drop(columns=['goals/conceded','goals/saves','penalty/saved','league','season'])

#Y=(filtered_by_league_NULL_FW['rating']*100).astype('int') #For Classification

Y=filtered_by_league_NULL_FW['rating']

X.columns


Index(['captain', 'shots/total', 'shots/on', 'goals/total', 'goals/assists',
       'passes/total', 'passes/key', 'passes/accuracy', 'tackles/total',
       'tackles/blocks', 'tackles/interceptions', 'duels/total', 'duels/won',
       'dribbles/attempts', 'dribbles/success', 'fouls/drawn',
       'fouls/committed', 'cards/yellow', 'cards/yellowred', 'cards/red',
       'penalty/won', 'penalty/commited', 'penalty/success', 'penalty/missed',
       'games/appearences', 'games/minutes_played', 'games/lineups',
       'substitutes/in'],
      dtype='object')

### Spliting the data into Train & Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [ ]:
#for vesion only
filtered_by_league_NULL_FW[['rating']].describe()

,rating
count,111.000000
mean,6.843437
std,0.301589
min,6.300000
25%,6.650000
50%,6.793333
75%,6.939999
max,7.817647


### Scaling the feature vector

Let's scale the feature vector.

First the standard scaler substracts by the mean and divides by std.  Let's try that ourselves and see if that looks right.

In [ ]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
#y_train = sc_y.fit_transform(y_train)

### Reducing Dimensions( **Whitening Data**)



In [ ]:
#for vesion only
X.corr()

,captain,shots/total,shots/on,goals/total,goals/assists,passes/total,passes/key,passes/accuracy,tackles/total,tackles/blocks,tackles/interceptions,duels/total,duels/won,dribbles/attempts,dribbles/success,fouls/drawn,fouls/committed,cards/yellow,cards/yellowred,cards/red,penalty/won,penalty/commited,penalty/success,penalty/missed,games/appearences,games/minutes_played,games/lineups,substitutes/in
captain,1.000000,0.054397,0.051003,0.070525,-0.018599,0.089590,0.068225,-0.070347,0.054291,0.159498,0.071754,0.134610,0.103944,-0.062131,-0.069832,-0.003954,0.091451,0.064149,-0.022559,0.376174,-0.031732,0.248152,0.420643,0.606777,0.112599,0.120524,0.121696,0.003284
shots/total,0.054397,1.000000,0.970847,0.886684,0.680120,0.777338,0.773405,0.068668,0.572498,0.488724,0.572894,0.764730,0.729477,0.744787,0.698372,0.695476,0.671628,0.564745,0.102273,0.130119,0.324317,0.137471,0.464046,0.274307,0.791962,0.873093,0.871791,-0.010145
shots/on,0.051003,0.970847,1.000000,0.937763,0.674515,0.760806,0.779551,0.086622,0.493093,0.435479,0.486060,0.695820,0.658143,0.716962,0.690442,0.622867,0.613757,0.520249,0.077468,0.110337,0.313380,0.120727,0.499790,0.319733,0.759729,0.846038,0.850055,-0.038575
goals/total,0.070525,0.886684,0.937763,1.000000,0.628154,0.681370,0.697879,0.083891,0.399198,0.372718,0.369589,0.582269,0.538003,0.638076,0.610559,0.505162,0.532866,0.464376,0.130894,0.083165,0.325766,0.209919,0.572779,0.372973,0.705305,0.791881,0.795842,-0.049831
goals/assists,-0.018599,0.680120,0.674515,0.628154,1.000000,0.805466,0.795229,0.212422,0.466776,0.181997,0.537978,0.558253,0.555903,0.725942,0.701149,0.491724,0.456792,0.373017,0.172413,0.110148,0.338717,0.221802,0.212110,0.131180,0.604423,0.678081,0.683568,-0.045974
passes/total,0.089590,0.777338,0.760806,0.681370,0.805466,1.000000,0.935354,0.222933,0.672972,0.368511,0.717535,0.733769,0.733205,0.840462,0.842678,0.747239,0.638104,0.483804,0.147765,0.164585,0.288395,0.334130,0.420430,0.230711,0.769943,0.840659,0.850794,-0.016665
passes/key,0.068225,0.773405,0.779551,0.697879,0.795229,0.935354,1.000000,0.185758,0.557561,0.290904,0.560438,0.674596,0.685836,0.839845,0.861911,0.716135,0.573794,0.455000,0.104471,0.115097,0.358301,0.206161,0.455528,0.179567,0.708552,0.796519,0.805864,-0.063402
passes/accuracy,-0.070347,0.068668,0.086622,0.083891,0.212422,0.222933,0.185758,1.000000,0.077722,-0.103634,0.121870,-0.083924,-0.079959,0.218420,0.229290,0.075650,-0.099869,-0.133262,0.046956,-0.016468,0.010290,0.108671,0.044300,-0.022368,-0.042859,0.022168,0.052381,-0.208355
tackles/total,0.054291,0.572498,0.493093,0.399198,0.466776,0.672972,0.557561,0.077722,1.000000,0.476493,0.876098,0.755655,0.722101,0.618724,0.560514,0.725611,0.730421,0.459951,-0.008730,0.275619,0.208863,0.108869,0.242176,0.085293,0.655080,0.721686,0.729306,-0.025585
tackles/blocks,0.159498,0.488724,0.435479,0.372718,0.181997,0.368511,0.290904,-0.103634,0.476493,1.000000,0.424673,0.547636,0.484620,0.238022,0.191287,0.475471,0.632849,0.537161,0.005405,0.169000,0.045619,0.205402,0.296093,0.278726,0.488825,0.507092,0.499626,0.074463


###Because the stats are **Very heavily Correlated**

May Lead ** Biasing ! **
*  Reduces training time — due to smaller dataset
*  Removes noise — by keeping only what’s relevant


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_r = pca.fit(X_train).transform(X_train)

**The components are the two axes of the projected back into the original features space.
We can use these to calculate the relative components of the original features in our 
new features.**


In [ ]:
# Let's get the top components of PC1:
print("top components of Principal Component 1(Eigenvector):")

#SS (Sum 0f square distances)
SS_Distances_PC1 = np.abs(pca.components_[0])/np.sum(np.abs(pca.components_[0])) # rel_values
print("Feature Names: " + str([features[i] for i in np.argsort(-SS_Distances_PC1)[:5]]))
print("Percentages: " + str(SS_Distances_PC1[np.argsort(-SS_Distances_PC1)[:5]]))

top components of Principal Component 1(Eigenvector):
Feature Names: ['games/minutes_played', 'games/lineups', 'duels/total', 'passes/total', 'shots/total']
Percentages: [0.05387218 0.05370115 0.05089352 0.05033281 0.05005916]


In [ ]:
# Let's get the top components of PC2:
print("top components of Principal Component 2 :")

SS_Distances_PC2 = np.abs(pca.components_[1])/np.sum(np.abs(pca.components_[1]))
print("Feature Names: " + str([features[i] for i in np.argsort(-SS_Distances_PC2)[:3]]))
print("Percentages: " + str(SS_Distances_PC2[np.argsort(-SS_Distances_PC2)[:3]]))

top components of Principal Component 2 :
Feature Names: ['captain', 'penalty/missed', 'penalty/commited']
Percentages: [0.1304652  0.12832948 0.09988909]



# Linear Regression in Sklearn 

### Overview
Instructor to demo this on screen.
 
### Builds on
None

### Run time
approx. 20-30 minutes

In [ ]:
import statsmodels.formula.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Create linear regression object
lin_reg_mod = linear_model.LinearRegression()
lin_reg_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred = lin_reg_mod.predict(X_test)
#filtered_by_league_NULL_FW['New_Rate']=lin_reg_mod.predict(X)

#Evaluation Of The Model


We'll now check the predictions against the actual values by using the RMSE and R-2 metrics, two metrics commonly used to evaluate regression tasks:

In [ ]:
Result = pd.DataFrame({'Actual Rate':y_test ,'Predicted_Rate':y_pred})
Result

,Actual Rate,Predicted_Rate
1881,6.658333,6.660760
879,6.655555,6.663060
1085,6.742857,6.643975
1612,6.725000,6.697518
1470,7.041666,7.329397
2101,6.861111,6.599849
2044,6.800000,7.007221
1103,6.525000,6.657147
617,6.872727,6.777634
720,6.771875,6.573388


In [ ]:
test_set_rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

test_set_r2 = r2_score(y_test, y_pred)

In [ ]:
print('The Error percentage(RMSE) in our model = {}'.format(test_set_rmse))
#print(test_set_r2)

The Error percentage(RMSE) in our model = 0.18487695014819827


In [ ]:
!pip install eli5

     |████████████████████████████████| 112kB 7.7MB/s 


In [ ]:
import eli5
eli5.explain_weights(lin_reg_mod, feature_names=list(X.columns))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight?,Feature
+6.852,<BIAS>
+0.244,games/minutes_played
+0.231,dribbles/success
+0.230,passes/key
+0.157,shots/total
+0.152,duels/won
+0.149,goals/total
+0.046,tackles/total
+0.045,tackles/blocks
+0.017,goals/assists


In [ ]:
# Building the optimal model using Backward Elimination
import statsmodels.api as sm
X = np.append(arr = np.ones((465, 1)).astype(int), values = X, axis = 1)
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit() #ordinary least
regressor_OLS.summary()
X_opt = X[:, [0, 1, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3, 5]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X[:, [0, 3]]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
regressor_OLS.summary()

ValueError: ignored

#Building a new Dataframe
## Filtering the Data

In [ ]:
Tested_Data=Full_Data[Full_Data.league=='Premier League'].copy()
Tested_Data=Tested_Data[Tested_Data.rating>0]
ready_for_test= Tested_Data.iloc[:,20:53].drop(columns=['goals/conceded','goals/saves','penalty/saved','league','season'])
#prepare the data
sc_data = StandardScaler()
X_test_Data = sc_data.fit_transform(ready_for_test)
#train
new = lin_reg_mod.predict(X_test_Data)
Tested_Data.head(1)

,Unnamed: 0,Unnamed: 1,player_id,player_name,firstname,lastname,number,position,age,birth_date,birth_place,birth_country,nationality,height,weight,injured,rating,team_id,team_name,league_id,league,season,captain,shots/total,shots/on,goals/total,goals/conceded,goals/assists,goals/saves,passes/total,passes/key,passes/accuracy,tackles/total,tackles/blocks,tackles/interceptions,duels/total,duels/won,dribbles/attempts,dribbles/success,fouls/drawn,fouls/committed,cards/yellow,cards/yellowred,cards/red,penalty/won,penalty/commited,penalty/success,penalty/missed,penalty/saved,games/appearences,games/minutes_played,games/lineups,substitutes/in,substitutes/out,substitutes/bench
1,NaN,1,1436,Petr Čech,Petr,Čech,NaN,Goalkeeper,37,20/05/1982,Plzeň,Czech Republic,Czech Republic,196 cm,90 kg,NaN,6.855882,42,Arsenal,37,Premier League,2017-2018,10,0,0,0,48,0,91,530,0,66,0,0,0,23,21,3,2,4,0,1,0,0,0,0,0,0,1,34,3040,34,0,1,1


In [ ]:
#intialzie once
#Tested_Data['Our_rating']=new  #oldmethod
Tested_Data.insert(15,'Our_rating',new)

In [ ]:
Tested_Data.head(1)

,Unnamed: 0,Unnamed: 1,player_id,player_name,firstname,lastname,number,position,age,birth_date,birth_place,birth_country,nationality,height,weight,Our_rating,injured,rating,team_id,team_name,league_id,league,season,captain,shots/total,shots/on,goals/total,goals/conceded,goals/assists,goals/saves,passes/total,passes/key,passes/accuracy,tackles/total,tackles/blocks,tackles/interceptions,duels/total,duels/won,dribbles/attempts,dribbles/success,fouls/drawn,fouls/committed,cards/yellow,cards/yellowred,cards/red,penalty/won,penalty/commited,penalty/success,penalty/missed,penalty/saved,games/appearences,games/minutes_played,games/lineups,substitutes/in,substitutes/out,substitutes/bench
1,NaN,1,1436,Petr Čech,Petr,Čech,NaN,Goalkeeper,37,20/05/1982,Plzeň,Czech Republic,Czech Republic,196 cm,90 kg,6.560301,NaN,6.855882,42,Arsenal,37,Premier League,2017-2018,10,0,0,0,48,0,91,530,0,66,0,0,0,23,21,3,2,4,0,1,0,0,0,0,0,0,1,34,3040,34,0,1,1


In [ ]:

#visualize the data 
Tested_Data[['rating','Our_rating']]

,rating,Our_rating
1,6.855882,6.560301
9,6.865000,6.937672
17,7.224000,6.728081
33,7.715789,7.614971
37,7.318181,6.870524
...,...,...
2270,6.917241,6.851402
2273,6.800000,6.636177
2276,6.708333,6.711335
2279,6.947368,6.651566


# Visualizng the Hypthosis

In [ ]:
! pip install shapash

     |████████████████████████████████| 849kB 7.7MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 327kB 16.0MB/s 
     |████████████████████████████████| 13.1MB 238kB/s 
     |████████████████████████████████| 1.8MB 52.6MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 645kB 35.4MB/s 
     |████████████████████████████████| 194kB 46.8MB/s 
     |████████████████████████████████| 1.0MB 39.7MB/s 
     |████████████████████████████████| 3.5MB 43.3MB/s 
     |████████████████████████████████| 358kB 34.3MB/s 
  Created wheel for dash-bootstrap-components: filename=dash_bootstrap_components-0.9.1-cp37-none-any.whl size=170749 sha256=fe320ecf46f45b1a837350d1b20a74bd00e80e3670b7cbf59d8f074250b8f188
  Stored in directory: /root/.cache/pip/wheels/56/25/46/7cbcb9c004b9acf8

In [ ]:
#prepare to the shapash
features_dict =dict()
for x in features :
  features_dict[x]=str(x+'_new')
features_dict

{'captain': 'captain_new',
 'cards/red': 'cards/red_new',
 'cards/yellow': 'cards/yellow_new',
 'cards/yellowred': 'cards/yellowred_new',
 'dribbles/attempts': 'dribbles/attempts_new',
 'dribbles/success': 'dribbles/success_new',
 'duels/total': 'duels/total_new',
 'duels/won': 'duels/won_new',
 'fouls/committed': 'fouls/committed_new',
 'fouls/drawn': 'fouls/drawn_new',
 'games/appearences': 'games/appearences_new',
 'games/lineups': 'games/lineups_new',
 'games/minutes_played': 'games/minutes_played_new',
 'goals/assists': 'goals/assists_new',
 'goals/total': 'goals/total_new',
 'passes/accuracy': 'passes/accuracy_new',
 'passes/key': 'passes/key_new',
 'passes/total': 'passes/total_new',
 'penalty/commited': 'penalty/commited_new',
 'penalty/missed': 'penalty/missed_new',
 'penalty/success': 'penalty/success_new',
 'penalty/won': 'penalty/won_new',
 'players__age': 'players__age_new',
 'players__birth_country': 'players__birth_country_new',
 'players__birth_date': 'players__birth_da

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer
xpl = SmartExplainer(features_dict=features_dict)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)
XXX=X_test.reset_index()
XXX=XXX.drop(columns='index')


In [ ]:
ypred=pd.DataFrame(lin_reg_mod.predict(X_test), columns=['pred'])


In [ ]:

xpl.compile(x=XXX,model=lin_reg_mod
    #preprocessing=encoder, # Optional: compile step can use inverse_transform method
    ,y_pred=ypred # Optional
    #postprocessing=postprocess # Optional: see tutorial postprocessing
)

Backend: Shap LinearExplainer


/usr/local/lib/python3.7/dist-packages/shap/explainers/_linear.py:65: UserWarning:

The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, or maskers.Impute)



In [ ]:
app = xpl.run_app()

Dash is running on http://0.0.0.0:8050/



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://7d1a1eb23364:8050/
INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)


In [ ]:
predictor = xpl.to_smartpredictor()

ValueError: ignored

# Exporting the file

In [ ]:
Tested_Data.to_csv(r'Data with out model.csv', index=False)

In [ ]:
!ls

 all_Data.xlsx	'Data with out model.csv'   sample_data


In [ ]:
! cat all_Data_all_the_time.xlsx

cat: all_Data_all_the_time.xlsx: No such file or directory


In [ ]:
#Save the DataFrame into Local 
Tested_Data.to_excel(r'C:\Users\Abdo\Desktop\Graduation project\Data For Graduation\Data with out model.xlsx', index=False)